# 사용자 맞춤 뉴스 추천 모델링

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_bosa = pd.read_csv("./csv/bosa_news.csv")
df_bosa.loc[2369] # 존재하지 않는 링크이므로 삭제 조치

_id                                    65e6bfc43cd41b2567398ad6
news_title                                                  NaN
news_url      http://www.bosa.co.kr/news/articleView.html?id...
news_when                                      2004.01.07 09:02
news_topic                                                   병원
Name: 2369, dtype: object

In [3]:
df_bosa.drop(columns=['_id'], inplace=True)
df_bosa.dropna(subset=['news_title'], inplace=True)
df_bosa.head()

,news_title,news_url,news_when,news_topic
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,02.29 11:57,다국적제약/의료기기
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:52,제약산업
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:29,동정
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:07,제약산업
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,02.29 05:50,학회/학술


In [4]:
df_bosa['news_topic'].value_counts()  #제약/개발 #병원 #법령/정책 #의료기기 #학회/학술 #

news_topic
제약산업          472
제약            360
기타기관          251
병원            212
병의원           206
복지부            95
기획연재           91
제약·유통          67
다국적제약/의료기기     62
국회             61
학회/학술          60
개원가            57
식약처            54
바이오            52
동정             47
의료             41
의료기기·IT        40
질병청            37
심평원            33
의약정책           19
건보공단           19
의료단체           16
약사단체           12
유통             10
식품              8
협회              8
진흥원             5
책소개             5
포토뉴스            4
인사              4
단신              3
정책·행정           3
의료기기            3
약사·약국           2
너몰내알            1
보사연             1
치과/한의사          1
의원·병원           1
화장품             1
약대/학술           1
Name: count, dtype: int64

In [5]:
df_bosa[df_bosa.isnull().any(axis=1)] # 위에서 삭제 조치 했음

,news_title,news_url,news_when,news_topic


## 시도1 : 군집화 후 토픽 분류
- 우선 1차적으로 군집을 만들어 어떠한 topic으로 분류되는지 확인해보자

In [6]:
df_bosa_test1 = df_bosa.copy()

### 토큰화

In [45]:
# 불용어 리스트 생성
stopwords = ['희귀질환', '한다', '가다', '맞다', '대다', '캐다', '희귀', '건보', '케다']
from konlpy.tag import Okt
okt = Okt()

#토크나이징 함수 정의
def tokenizer(raw, pos=['Noun', 'Verb'], stopword=stopwords):
    return [
        word for word, tag in okt.pos(raw, norm=True, stem=True)
        if len(word) >1 and tag in pos and word not in stopword
    ]

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=3)

In [47]:
features = tfidfvectorizer.fit_transform(df_bosa['news_title'])
# news_title에 있는 내용을 백터&토큰화

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [48]:
features

<2425x1043 sparse matrix of type '<class 'numpy.float64'>'
	with 11638 stored elements in Compressed Sparse Row format>

### 군집 만들기

In [49]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.decomposition import LatentDirichletAllocation

In [50]:
components_5 = LatentDirichletAllocation(n_components=5, n_jobs=-1)
components_5.fit(features)

LatentDirichletAllocation(n_components=5, n_jobs=-1)

In [51]:
dictionary_list = tfidfvectorizer.get_feature_names_out(features)
dictionary_list

array(['가격', '가능', '가능성', ..., '희귀난치성질환', '희망', '희소'], dtype=object)

In [52]:
components_5.components_ # 토픽별로 단어의 확률 분포를 나타냄

array([[0.20099154, 0.20216834, 3.40075298, ..., 0.20004249, 0.20065943,
        2.91294744],
       [0.23538132, 0.20052532, 1.83229681, ..., 0.20057147, 1.04192182,
        0.2000398 ],
       [3.70374105, 0.20131159, 0.20428785, ..., 0.20156278, 0.20080167,
        0.20042977],
       [0.20084678, 1.73105662, 0.62307626, ..., 0.20494253, 7.56060845,
        0.64002726],
       [0.20098832, 5.70177799, 0.20152755, ..., 8.96409853, 1.82044663,
        0.20004604]])

In [53]:
topics_output = components_5.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [54]:
df_topics_score

,0,1,2,3,4
0,0.062869,0.250919,0.064526,0.062692,0.558995
1,0.060322,0.059707,0.060746,0.059470,0.759755
2,0.055830,0.055710,0.773411,0.059368,0.055680
3,0.068807,0.069676,0.367273,0.068644,0.425600
4,0.055870,0.055918,0.057974,0.773922,0.056317
...,...,...,...,...,...
2420,0.085525,0.659470,0.084741,0.084593,0.085671
2421,0.076452,0.719126,0.067007,0.069659,0.067757
2422,0.418302,0.391639,0.063212,0.063014,0.063833
2423,0.084070,0.659134,0.083914,0.086064,0.086817


In [55]:
import numpy as np
df_topics_score['topic_num'] = np.argmax(topics_output, axis=1)

In [56]:
df_topics_score['news_title'] = df_bosa['news_title']
df_topics_score

,0,1,2,3,4,topic_num,news_title
0,0.062869,0.250919,0.064526,0.062692,0.558995,4,"한국쿄와기린, 세계 희귀질환의 날 행사 참여"
1,0.060322,0.059707,0.060746,0.059470,0.759755,4,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최"
2,0.055830,0.055710,0.773411,0.059368,0.055680,2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상"
3,0.068807,0.069676,0.367273,0.068644,0.425600,4,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시"
4,0.055870,0.055918,0.057974,0.773922,0.056317,3,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다
...,...,...,...,...,...,...,...
2420,0.085525,0.659470,0.084741,0.084593,0.085671,1,쇼그렌증후군 관여 단백질 발견
2421,0.076452,0.719126,0.067007,0.069659,0.067757,1,26일 `희귀질환 치료' 심포지엄
2422,0.418302,0.391639,0.063212,0.063014,0.063833,0,`희귀^난치성질환자 연합회' 구성 추진
2423,0.084070,0.659134,0.083914,0.086064,0.086817,1,책임운영기관 평가 “적정성 결여


In [57]:
# 그럼 이제 각각의 news가 어떤 topic으로 분류되는지는 알았는데
# 각 토픽이 어떤 단어를 가지고 있는지 알기 위해서는 상위 단어 추출이 필요함

# 상위 단어 추출

topics_list = []
for topic in components_5.components_ : # 토픽별로 단어의 확률 분포 나타낸 것
    df_datas = [topic, dictionary_list] # dictionary_list는 단어 나열
    df_topics = pd.DataFrame(data=df_datas).T
    df_topics = df_topics.sort_values(0, ascending=False)
    topics_text = ' '.join(df_topics[1].values[:5]) # 상위 4개를 시리즈 형식으로 출력하기
    print(topics_text) 
    topics_list.append(topics_text)

topics_list_table = [topics_list, ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5']]
topics_list_table

치료 지정 의약품 확대 급여
치료 개발 유전자 제휴 스케
신약 승인 시장 제약 생명
지원 환자 의료 관리 의약품
개최 치료 증후군 진단 샤이어


[['치료 지정 의약품 확대 급여',
  '치료 개발 유전자 제휴 스케',
  '신약 승인 시장 제약 생명',
  '지원 환자 의료 관리 의약품',
  '개최 치료 증후군 진단 샤이어'],
 ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5']]

In [58]:
topics_list_table = pd.DataFrame(topics_list_table)
topics_list_table

,0,1,2,3,4
0,치료 지정 의약품 확대 급여,치료 개발 유전자 제휴 스케,신약 승인 시장 제약 생명,지원 환자 의료 관리 의약품,개최 치료 증후군 진단 샤이어
1,Topic1,Topic2,Topic3,Topic4,Topic5


In [59]:
topics_list_table[0][0]

'치료 지정 의약품 확대 급여'

In [60]:
pivot_table_title = pd.pivot_table(data=df_topics_score,
                                   values='news_title'
                                   ,index='topic_num'
                                   , aggfunc='count')
pivot_table_title

,news_title
topic_num,
0,482
1,512
2,464
3,500
4,466


### 시각화

In [61]:
vis = pyLDAvis.lda_model.prepare(components_5, features, tfidfvectorizer)
# 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [62]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

In [23]:
df_bosa_test1['topic_model'] = df_topics_score['topic_num']
df_bosa_test1.head()

,news_title,news_url,news_when,news_topic,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,02.29 11:57,다국적제약/의료기기,0.0
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:52,제약산업,0.0
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:29,동정,1.0
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:07,제약산업,0.0
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,02.29 05:50,학회/학술,0.0


In [24]:
type(df_bosa_test1['topic_model'][0])

numpy.float64

In [25]:
for idx, topic in enumerate(df_bosa_test1['topic_model']) :
    if topic == 0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[0][0]
    elif topic == 1.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[1][0]
    elif topic == 2.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[2][0]
    elif topic == 3.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[3][0]
df_bosa_test1.head()

/tmp/ipykernel_955/1707612536.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '지정 신약 의약품 치료' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[0][0]


,news_title,news_url,news_when,news_topic,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,02.29 11:57,다국적제약/의료기기,지정 신약 의약품 치료
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:52,제약산업,지정 신약 의약품 치료
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:29,동정,의료 교수 생명 인하대병원
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:07,제약산업,지정 신약 의약품 치료
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,02.29 05:50,학회/학술,지정 신약 의약품 치료


## 시도2 : 전처리 후 다시 군집화하기

### news_topic 전처리

In [26]:
df_bosa.head()

,news_title,news_url,news_when,news_topic
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,02.29 11:57,다국적제약/의료기기
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:52,제약산업
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:29,동정
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:07,제약산업
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,02.29 05:50,학회/학술


In [27]:
df_bosa['news_topic'].value_counts()

news_topic
제약산업          472
제약            360
기타기관          251
병원            212
병의원           206
복지부            95
기획연재           91
제약·유통          67
다국적제약/의료기기     62
국회             61
학회/학술          60
개원가            57
식약처            54
바이오            52
동정             47
의료             41
의료기기·IT        40
질병청            37
심평원            33
의약정책           19
건보공단           19
의료단체           16
약사단체           12
유통             10
식품              8
협회              8
진흥원             5
책소개             5
포토뉴스            4
인사              4
단신              3
정책·행정           3
의료기기            3
약사·약국           2
너몰내알            1
보사연             1
치과/한의사          1
의원·병원           1
화장품             1
약대/학술           1
Name: count, dtype: int64

In [29]:
df_bosa.loc[df_bosa['news_topic'] == '기타기관']

,news_title,news_url,news_when,news_topic
113,"NECA, 제2차 '제한적 의료기술' 신청 접수 재공고",http://www.bosa.co.kr/news/articleView.html?id...,2023.11.10 09:56,기타기관
174,"보의연, 2023년 제2차 ‘제한적 의료기술’ 신청 접수",http://www.bosa.co.kr/news/articleView.html?id...,2023.09.27 17:02,기타기관
262,"NECA, ‘2023년 1차 제한적 의료기술’ 신청접수",http://www.bosa.co.kr/news/articleView.html?id...,2023.06.05 10:03,기타기관
344,보건의료 200여개 법령 ‘읽기 쉬운 문장’ 개선 추진,http://www.bosa.co.kr/news/articleView.html?id...,2023.03.17 06:00,기타기관
348,"케이메디, KAIST 희귀질환 파브리병 치료제 개발 연구 지원",http://www.bosa.co.kr/news/articleView.html?id...,2023.03.14 14:53,기타기관
...,...,...,...,...
2406,만성·희귀질환자 등 의료급여 확대,http://www.bosa.co.kr/news/articleView.html?id...,2002.05.20 10:38,기타기관
2410,고혈압 등 10개질환 급여일수 30일 연장,http://www.bosa.co.kr/news/articleView.html?id...,2001.12.12 11:33,기타기관
2413,혈소판감소성 자반증치료제 희귀약 지정,http://www.bosa.co.kr/news/articleView.html?id...,2001.11.06 10:50,기타기관
2415,“희귀의약품센터는 예산 은닉처(?)”,http://www.bosa.co.kr/news/articleView.html?id...,2001.09.11 12:04,기타기관


In [63]:
df_bosa.all

<bound method DataFrame.all of                                  news_title  \
0                  한국쿄와기린, 세계 희귀질환의 날 행사 참여   
1        광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최   
2          순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상   
3          창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시   
4     [세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다   
...                                     ...   
2421                     26일 `희귀질환 치료' 심포지엄   
2422                  `희귀^난치성질환자 연합회' 구성 추진   
2423                      책임운영기관 평가 “적정성 결여   
2424                    먹지도 굶지도 못하는 희귀질환 발견   
2425                희귀의약품센터 공익성 기부금 대상단체 지정   

                                               news_url         news_when  \
0     http://www.bosa.co.kr/news/articleView.html?id...       02.29 11:57   
1     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:52   
2     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:29   
3     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:07   
4     http://www.bo